<a href="https://colab.research.google.com/github/Minwoo-study/Korean_Classic_DB/blob/main/%ED%95%9C%EA%B5%AD%EA%B3%A0%EC%A0%84%EC%A2%85%ED%95%A9DB_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국고전종합DB 원문 번역문 word2vec

In [ ]:
import pandas as pd
# import pickle5 as pickle
import pickle
import re
import regex
from tqdm import tqdm
tqdm.pandas()

from kiwipiepy import Kiwi, Option
kiwi = Kiwi()
kiwi.prepare()

from kss import split_sentences
import itertools
from collections import defaultdict
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib as mpl
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='NanumBarunGothic')
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
import matplotlib.colors as clr

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### 번역문 전처리

In [ ]:
# 전처리 함수화
# def preprocess(title,text):
#     text = re.sub(f'\n{title}\n\n<sep>','',text)
#     text = re.sub(f'^\n{title}\n\n','',text)
#     text = re.sub('<sep>\n\n$','',text)
#     text = re.sub('\n',' ',text)
#     text = re.sub('\=+ \d{1,} \=+(\<sep\>)*','',text)
    
#     #한자제거 추가
#     text = re.sub(r'\([\W]*\)', '', text)
#     text = re.sub('[一-龥]', '', text)
#     text = re.sub("[()]","", text)
    
#     text = regex.sub('\=+ \p{Hangul}+ \=+(\<sep\>)*','',text)
#     text = regex.sub('\=+ \p{Han}+ \=+(\<sep\>)*','',text)
#     text = text.strip()
#     text = re.sub('<sep>\n*$','',text)
#     text_list = text.split('<sep>')
#     text_list = [x.strip() for x in text_list if not re.match('\=+.+\=+',x)]
#     # text_list = [x for x in text_list if not re.match('\=+.+\=+',x)]
#     return text_list

In [ ]:
def preprocess(text):
    text = re.sub('\n',' ',text).strip()
    text = re.sub("[()《》〔〕]","", text)
    text = re.sub('[一-龥]', '', text)
    text = regex.sub('\(\p{Han}+\)','',text)
    return text

In [ ]:
preprocess(data['번역문'][0])

'예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리에 놓는 것은 물이 모든 맛의 근본임을 중시하려고 해서이다. 각종 화려한 무늬의 옷감이 아름답기는 하지만 제사 때에 거친 삼베로 동이를 덮는 것은 부녀자들이 하는 길쌈질의 시초를 돌아보고 귀하게 여기려 해서이다.     ”라고 하였다. 옛날의 군자가 그 근본과 시초를 반드시 중하게 여기려 한 것이 이와 같았다.우리 동방에 문장이 나와서 글을 지어 후세에 전할 수 있게 된 것은 고운 최공으로부터 비롯되었다. 그리고 우리 동방의 선비로서 북쪽으로 중국에 유학하여 문장으로 천하에 명성을 떨친 것도 최공으로부터 시작되었다. 최공의 글 가운데 후세에 전하는 것으로는 오직 계원필경과 중산복궤집 2부가 있을 뿐이다. 그러고 보면 이 2부의 서책이 또한 우리 동방 문장의 근본이요 시초라고 할 것이다.글을 숭상하는 우리 동방의 풍조가 아조에 이르러서는 더욱 빛나고 무르익어 집집마다 연허와 조유가 배출되면서 시와 문으로 문집을 이룬 것이 집을 가득 채울 정도로 엄청나게 많았다. 하지만 최공의 글을 아는 사람은 드물기만 하였다.내가 일찍이 근대 사람이 편찬한 동국서목을 보니 중산복궤집이 실려 있기에 널리 구해 보았지만 끝내 얻지 못하였다. 다만 계원필경 20권은 우리 집안에서 대대로 소장해 왔으므로 내가 어려서부터 진귀한 글로 알고서 음미해 왔다. 그러나 간혹 사람들과 이야기를 나누다 보면, 비록 박아하고 글을 잘하며 옛것을 좋아하는 자라고 할지라도 모두 그 글을 본 적이 없다고 말하고 있으니, 그러고 보면 이 글이 거의 없어질 지경에 이르렀다고 해야 할 것이다. 만약 이 글이 동방에 전하지 않게 된다면, 이는 현주를 태실에 진설하지 않는 것과 같고, 거친 삼베로 제사용 술동이를 덮지 않는 것과 같게 될 것이니, 어떻게 백성에게 근본을 잊지 말라고 가르칠 수가 있겠는가.세상에서는 간혹 공의 글이 모두 변려 사륙문으로서 옛 작자의 글과는 완전히 다르다고 비평하기도 한다. 그러나 공이 중국에 들어가서

In [ ]:
data=data.reset_index(drop=True)

In [ ]:
data['text_list'] = data.progress_apply(lambda x:preprocess(x['번역문']),axis=1)
data['text_list']

100%|██████████| 104127/104127 [00:05<00:00, 17671.18it/s]
/tmp/ipykernel_649961/1663847976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text_list'] = data.progress_apply(lambda x:preprocess(x['번역문']),axis=1)


0         예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...
1         계원필경집 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로나 사적...
2         회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 시어...
3         신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살펴보...
4         신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 돌아...
                                ...                        
105619    공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 휘준...
105620    부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 휘 ...
105621    이것은 우리 향산공의 수고로, 인출편이라고 하는 것이다. 공이 순국하신 지 6년이 ...
105622    글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을 ...
105623    오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...
Name: text_list, Length: 104127, dtype: object

In [ ]:
# data_dup['text_list'] = data_dup['text_list'].astype(str)

In [ ]:
data['text_list']

0         예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...
1         계원필경집 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로나 사적...
2         회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 시어...
3         신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살펴보...
4         신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 돌아...
                                ...                        
105619    공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 휘준...
105620    부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 휘 ...
105621    이것은 우리 향산공의 수고로, 인출편이라고 하는 것이다. 공이 순국하신 지 6년이 ...
105622    글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을 ...
105623    오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...
Name: text_list, Length: 104127, dtype: object

##### 문단 -> 문장

In [ ]:
# 문단 -> 문장 (오래걸림)
# data_dup['sentence'] = data_dup['text_list'].progress_map(lambda x:split_sentences(x, backend='mecab', disable_gc=True))
# data_dup['sentence']

In [ ]:
# 문단 -> 문장 (마침표 기준으로 분할)
data['sentence'] = data['text_list'].progress_map(lambda x:x.split('.'))
data['sentence']

100%|██████████| 104127/104127 [00:00<00:00, 199354.79it/s]
/tmp/ipykernel_649961/3797538806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentence'] = data['text_list'].progress_map(lambda x:x.split('.'))


0         [예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자...
1         [계원필경집 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로나 사...
2         [회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 시...
3         [신 모는 아룁니다, 금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...
4         [신 모는 아룁니다, 신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...
                                ...                        
105619    [공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다,  대사성 ...
105620    [부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다,  시조 ...
105621    [이것은 우리 향산공의 수고로, 인출편이라고 하는 것이다,  공이 순국하신 지 6년...
105622    [글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을...
105623    [오호라, 이 책은 우리 향산공의 유문이다,  공의 큰 절의가 이미 하늘에 있는 해...
Name: sentence, Length: 104127, dtype: object

In [ ]:
data = data.explode(['sentence']).reset_index(drop=True)

#### 형태소 분석


In [ ]:
## 토크나이징 함수 선언 (품사 포함)
def tokenize(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word + ('다' if tag.startswith('V') else '')+ '/'+ tag # 동사/형용사에는 '다'를 붙여줌 + / 품사
            for word, tag, _, _ in res
            if not tag.startswith('E') and not tag.startswith('J') and not tag.startswith('S') and word not in stopwords] # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거

In [ ]:
kiwi.add_user_word('해동', 'NNG', 100000)

False

In [ ]:
kiwi.analyze('해동에서는')

[([Token(form='해동', tag='NNP', start=0, len=2),
   Token(form='에서', tag='JKB', start=2, len=2),
   Token(form='는', tag='JX', start=4, len=1)],
  -24.13312530517578)]

In [ ]:
token_list = []
stopwords = ['이다','하다','있다','위하']
for i,result in enumerate(kiwi.analyze(data['sentence'].tolist())):
      if i % 200000 ==0:
            print(i)
      try:
            res, score = result[0]
            token_list.append([word + ('다' if tag.startswith('V') else '')+ '/'+ tag # 동사/형용사에는 '다'를 붙여줌 + / 품사
            for word, tag, _, _ in res
            if not tag.startswith('E') and not tag.startswith('J') and not tag.startswith('S') and word not in stopwords]) # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거)
      except UnicodeDecodeError:
            print('UnicodeDecodeError')
            token_list.append([])

0
200000
400000
600000
UnicodeDecodeError
800000
1000000
1200000
1400000


In [ ]:
data['token'] = pd.Series(token_list)
data['token']

0          [예기/NNG, 이르다/VV, 달다/VA, 술/NNG, 맛/NNG, 좋다/VA, 하...
1          [각종/NNG, 화려/XR, 하/XSA, 무늬/NNG, 옷감/NNG, 아름답다/VA...
2                                                    [하다/VV]
3          [옛날/NNG, 군자/NNG, 그/MM, 근본/NNG, 시초/NNG, 반드시/MAG...
4          [우리/NP, 동방/NNG, 문장/NNG, 나오다/VV, 글/NNG, 짓다/VV, ...
                                 ...                        
1584494    [만년/NNG, 이르다/VV, 옛/MM, 경전/NNG, 안다/VV, 깊다/VA, 산...
1584495    [경술년/NNG, 변고/NNG, 지금/NNG, 여/XSN, 년/NNB, 동안/NNG...
1584496    [다만/MAG, 유집/NNP, 어루만지다/VV, 길다/VA, 탄식/NNG, 하/XS...
1584497          [족/NNB, 질/XSN, 중/NNB, 균/NNG, 삼가/NNG, 쓰다/VV]
1584498                                                   []
Name: token, Length: 1584499, dtype: object

In [ ]:
#토큰 4개 이하인 것 확인
data[data['token'].map(lambda x:len(x)<=4)]

,ID,원문,번역문,text_list,sentence,token
2,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,”라고 하였다,[하다/VV]
28,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,”라고 하였다,[하다/VV]
35,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,,[]
70,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,계원필경집 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로나 사적...,,[]
76,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 시어...",”라고 하였습니다,[하다/VV]
...,...,...,...,...,...,...
1584483,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을 ...",”라고 하는 것이다,"[하다/VV, 것/NNB, 이다/VCP]"
1584484,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을 ...",족손 강호가 적는다,"[족/NNB, 손/NNG, 강호/NNP, 적다/VA]"
1584485,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을 ...",,[]
1584489,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...","오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...","”라고 하였으니, 참으로 그러하다","[하다/VV, 참으로/MAG, 그러/XR, 하/XSA]"


In [ ]:
# 불용어 제거
stopwords = ['하다/VV','하/XSV','이다/VCP','하/XSA','하다/VV','있다/VV','않다/VX','없다/VA','하다/VX','되다/VV','있다/VX','아니다/VCN']

In [ ]:
data['token'] = data['token'].progress_map(lambda x:[w for w in x if not w in stopwords])

100%|██████████| 1584499/1584499 [00:11<00:00, 132153.20it/s]


In [ ]:
# 상위 n개 단어 확인
cnt = Counter(list(itertools.chain(*data['token'].tolist())))
cnt.most_common(30) # 상위 N개

[('것/NNB', 754672),
 ('들/XSN', 274292),
 ('말/NNG', 270483),
 ('그/MM', 226145),
 ('수/NNB', 219178),
 ('사람/NNG', 201718),
 ('일/NNG', 164703),
 ('이/MM', 163453),
 ('되/XSV', 155712),
 ('못하다/VX', 144413),
 ('이/NP', 139344),
 ('때/NNG', 131641),
 ('한/MM', 128068),
 ('공/NNG', 127014),
 ('나/NP', 112356),
 ('같다/VA', 108238),
 ('마음/NNG', 106520),
 ('년/NNB', 104873),
 ('지다/VX', 104355),
 ('어찌/MAG', 100327),
 ('보다/VV', 97486),
 ('모두/MAG', 94735),
 ('히/NNG', 92737),
 ('알다/VV', 89360),
 ('뜻/NNG', 87490),
 ('주다/VX', 86557),
 ('또/MAG', 86167),
 ('생각/NNG', 80947),
 ('자/NNB', 80238),
 ('이르다/VV', 72754)]

In [ ]:
# 토큰 5개 이상만 활용
data = data[data['token'].map(lambda x:len(x)>=5)]

In [ ]:
data = data.reset_index(drop=True)
data

,ID,원문,번역문,text_list,sentence,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,"[예기/NNG, 이르다/VV, 달다/VA, 술/NNG, 맛/NNG, 좋다/VA, 제..."
1,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,각종 화려한 무늬의 옷감이 아름답기는 하지만 제사 때에 거친 삼베로 동이를 덮는 ...,"[각종/NNG, 화려/XR, 무늬/NNG, 옷감/NNG, 아름답다/VA, 제사/NN..."
2,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,옛날의 군자가 그 근본과 시초를 반드시 중하게 여기려 한 것이 이와 같았다,"[옛날/NNG, 군자/NNG, 그/MM, 근본/NNG, 시초/NNG, 반드시/MAG..."
3,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,우리 동방에 문장이 나와서 글을 지어 후세에 전할 수 있게 된 것은 고운 최공으로부...,"[우리/NP, 동방/NNG, 문장/NNG, 나오다/VV, 글/NNG, 짓다/VV, ..."
4,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,예기에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리...,그리고 우리 동방의 선비로서 북쪽으로 중국에 유학하여 문장으로 천하에 명성을 떨친...,"[그리고/MAJ, 우리/NP, 동방/NNG, 선비/NNG, 북쪽/NNG, 중국/NN..."
...,...,...,...,...,...,...
1377359,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...","오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...","공은 처음부터 학문을 하는 사람으로 자처하지 않고 천성 그대로 행하였으므로, 사람...","[공/NNG, 처음/NNG, 학문/NNG, 사람/NNG, 자처/NNG, 천성/NNG..."
1377360,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...","오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...",만년에 이르러서는 옛 경전을 안고 깊은 산으로 들어가 독실하게 공부하면서 감히 단...,"[만년/NNG, 이르다/VV, 옛/MM, 경전/NNG, 안다/VV, 깊다/VA, 산..."
1377361,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...","오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...","경술년의 변고로부터 지금까지 20여 년 동안, 인심과 세도가 나날이 더러운 데로 ...","[경술년/NNG, 변고/NNG, 지금/NNG, 여/XSN, 년/NNB, 동안/NNG..."
1377362,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...","오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와 ...",다만 유집을 어루만지며 길게 탄식할 뿐이다,"[다만/MAG, 유집/NNP, 어루만지다/VV, 길다/VA, 탄식/NNG, 뿐/NNB]"


### 번역문 word2vec

In [ ]:
# vector : 차원, window : 앞뒤 단어 보는 크기, min_count : 분석에 포함할 최소 단어 빈도수, sg : 0이면 cbow, 1이면 skip-gram
v300_w5_min300_sg = Word2Vec(sentences=data['token'].to_list(), vector_size=300, window=5, min_count=300,sg=1,seed=2021)
v300_w5_min300_sg.save('ko_v300_w5_min300_sg.model') # 모델 저장. 모델 로드할때는 model= Word2Vec.load('v200_w3_min300_sg.model')

In [ ]:
target_list = ['우리/NP','나라/NNG','우리나라/NNG','동방/NNG','동국/NNG','삼한/NNP','조선/NNP']

In [ ]:
res = pd.DataFrame()
for target in target_list:
    res = pd.concat([res,pd.DataFrame(v300_w5_min300_sg.wv.most_similar(target, topn=20),columns=[target,'similarity'])],axis=1)  # get other similar words
res

,우리/NP,similarity,나라/NNG,similarity,우리나라/NNG,similarity,동방/NNG,similarity,동국/NNG,similarity,삼한/NNP,similarity,조선/NNP,similarity
0,저희/NP,0.555400,국가/NNG,0.575042,중국/NNP,0.611080,동국/NNG,0.619286,동방/NNG,0.619286,신라/NNP,0.552532,고려/NNP,0.552135
1,우리나라/NNG,0.525453,나라/NNP,0.515161,명나라/NNP,0.554832,우리나라/NNG,0.548760,중국/NNP,0.489839,고려/NNP,0.523660,국왕/NNG,0.542743
2,동방/NNG,0.502616,중국/NNP,0.476298,동방/NNG,0.548760,삼한/NNP,0.513429,해외/NNG,0.467793,동방/NNG,0.513429,일본/NNP,0.535210
3,너희/NP,0.423837,진나라/NNP,0.473287,조선/NNP,0.529667,조선/NNP,0.504681,우리나라/NNG,0.457415,개국/NNG,0.507930,우리나라/NNG,0.529667
4,중국/NNP,0.406554,동방/NNG,0.468282,우리/NP,0.525453,우리/NP,0.502616,삼한/NNP,0.454433,삼국/NNG,0.481479,중국/NNP,0.518314
5,조선/NNP,0.401422,말기/NNG,0.458797,소방/NNG,0.479751,중국/NNP,0.473456,조선/NNP,0.408096,조선/NNP,0.476945,대마도/NNP,0.515061
6,동국/NNG,0.388934,조선/NNP,0.456464,청나라/NNP,0.468132,나라/NNG,0.468282,일대/NNG,0.408055,태사/NNP,0.471619,명나라/NNP,0.508749
7,명나라/NNP,0.383103,우리나라/NNG,0.455680,일본/NNP,0.462605,문명/NNG,0.452482,지역/NNG,0.390062,고구려/NNP,0.456506,신라/NNP,0.507704
8,대국/NNG,0.374935,명나라/NNP,0.448192,동국/NNG,0.457415,도학/NNG,0.428002,우리/NP,0.388934,백제/NNP,0.456299,개국/NNG,0.507621
9,나/NP,0.350960,대국/NNG,0.440917,나라/NNG,0.455680,해외/NNG,0.426062,고장/NNG,0.386048,동국/NNG,0.454433,동방/NNG,0.504681


In [ ]:
res.to_excel('한국고전종합DB_번역문_word2vec.xlsx')

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('우리/NP', topn=20)  # get other similar words

[('저희/NP', 0.5554001927375793),
 ('우리나라/NNG', 0.525452733039856),
 ('동방/NNG', 0.5026158094406128),
 ('너희/NP', 0.42383748292922974),
 ('중국/NNP', 0.40655407309532166),
 ('조선/NNP', 0.4014221429824829),
 ('동국/NNG', 0.388933926820755),
 ('명나라/NNP', 0.38310277462005615),
 ('대국/NNG', 0.3749346435070038),
 ('나/NP', 0.3509595990180969),
 ('저/NP', 0.350872278213501),
 ('소방/NNG', 0.34709224104881287),
 ('열성/NNG', 0.34699350595474243),
 ('귀국/NNG', 0.33741137385368347),
 ('황조/NNP', 0.33727705478668213),
 ('선대/NNG', 0.33551546931266785),
 ('아/IC', 0.3328183591365814),
 ('운세/NNG', 0.33217090368270874),
 ('나라/NNG', 0.3300834000110626),
 ('보우/NNG', 0.32873275876045227)]

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('나라/NNG', topn=20)  # get other similar words

[('국가/NNG', 0.5750422477722168),
 ('나라/NNP', 0.5151606798171997),
 ('중국/NNP', 0.4762977957725525),
 ('진나라/NNP', 0.4732874035835266),
 ('동방/NNG', 0.4682823121547699),
 ('말기/NNG', 0.458796888589859),
 ('조선/NNP', 0.45646387338638306),
 ('우리나라/NNG', 0.45567983388900757),
 ('명나라/NNP', 0.44819194078445435),
 ('대국/NNG', 0.44091689586639404),
 ('왕실/NNG', 0.4408324062824249),
 ('당나라/NNP', 0.43115121126174927),
 ('주나라/NNP', 0.4241371750831604),
 ('위나라/NNP', 0.41377589106559753),
 ('송나라/NNP', 0.4052186906337738),
 ('멸망/NNG', 0.3988746404647827),
 ('초/NNP', 0.39514434337615967),
 ('상나라/NNP', 0.3926469087600708),
 ('강토/NNG', 0.3902271091938019),
 ('시대/NNG', 0.3889213502407074)]

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('우리나라/NNG', topn=20)  # get other similar words

[('중국/NNP', 0.6110798120498657),
 ('명나라/NNP', 0.5548319220542908),
 ('동방/NNG', 0.5487597584724426),
 ('조선/NNP', 0.5296672582626343),
 ('우리/NP', 0.5254528522491455),
 ('소방/NNG', 0.47975093126296997),
 ('청나라/NNP', 0.46813228726387024),
 ('일본/NNP', 0.4626047611236572),
 ('동국/NNG', 0.4574148654937744),
 ('나라/NNG', 0.45567983388900757),
 ('송나라/NNP', 0.44411179423332214),
 ('국조/NNG', 0.44326555728912354),
 ('역대/NNG', 0.44253021478652954),
 ('외국/NNG', 0.4414551258087158),
 ('황조/NNP', 0.4338394105434418),
 ('본국/NNG', 0.4303795397281647),
 ('오랑캐/NNG', 0.4283376932144165),
 ('금나라/NNP', 0.4206307530403137),
 ('대국/NNG', 0.4184335768222809),
 ('근세/NNG', 0.40758565068244934)]

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('동방/NNG', topn=20)  # get other similar words

[('동국/NNG', 0.6192862391471863),
 ('우리나라/NNG', 0.5487596988677979),
 ('삼한/NNP', 0.5134291052818298),
 ('조선/NNP', 0.5046807527542114),
 ('우리/NP', 0.5026158094406128),
 ('중국/NNP', 0.47345593571662903),
 ('나라/NNG', 0.4682822823524475),
 ('문명/NNG', 0.45248207449913025),
 ('도학/NNG', 0.42800185084342957),
 ('해외/NNG', 0.42606234550476074),
 ('문헌/NNG', 0.41064566373825073),
 ('기자/NNG', 0.4075033962726593),
 ('국조/NNG', 0.40189459919929504),
 ('위대/XR', 0.40034204721450806),
 ('문물/NNG', 0.39554229378700256),
 ('개국/NNG', 0.3948737680912018),
 ('고려/NNP', 0.38988378643989563),
 ('창도/NNG', 0.38861098885536194),
 ('문교/NNG', 0.38509392738342285),
 ('영남/NNP', 0.38195469975471497)]

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('동국/NNG', topn=20)  # get other similar words

[('동방/NNG', 0.619286298751831),
 ('중국/NNP', 0.4898388981819153),
 ('해외/NNG', 0.4677925705909729),
 ('우리나라/NNG', 0.4574148654937744),
 ('삼한/NNP', 0.4544330835342407),
 ('조선/NNP', 0.4080956280231476),
 ('일대/NNG', 0.40805482864379883),
 ('지역/NNG', 0.3900620937347412),
 ('우리/NP', 0.38893386721611023),
 ('고장/NNG', 0.38604849576950073),
 ('명나라/NNP', 0.38536667823791504),
 ('문헌/NNG', 0.3853549659252167),
 ('남방/NNG', 0.38213708996772766),
 ('국조/NNG', 0.37949904799461365),
 ('동남/NNG', 0.37889474630355835),
 ('왕조/NNG', 0.3767855763435364),
 ('삼국/NNG', 0.3653113543987274),
 ('영남/NNP', 0.36458665132522583),
 ('나라/NNG', 0.3608134388923645),
 ('지방/NNG', 0.3546033799648285)]

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('삼한/NNP', topn=20)  # get other similar words

[('신라/NNP', 0.5525323152542114),
 ('고려/NNP', 0.5236599445343018),
 ('동방/NNG', 0.5134291052818298),
 ('개국/NNG', 0.5079298615455627),
 ('삼국/NNG', 0.48147884011268616),
 ('조선/NNP', 0.4769449830055237),
 ('태사/NNP', 0.47161865234375),
 ('고구려/NNP', 0.45650598406791687),
 ('백제/NNP', 0.45629942417144775),
 ('동국/NNG', 0.45443305373191833),
 ('중원/NNP', 0.45373252034187317),
 ('태조/NNP', 0.4404228925704956),
 ('통일/NNG', 0.4404096305370331),
 ('계림/NNP', 0.43856188654899597),
 ('시조/NNG', 0.43188127875328064),
 ('통합/NNG', 0.43062376976013184),
 ('혁혁/XR', 0.42426738142967224),
 ('공훈/NNG', 0.4196465313434601),
 ('후예/NNG', 0.40516576170921326),
 ('고려조/NNG', 0.39854133129119873)]

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('조선/NNP', topn=20)  # get other similar words

[('고려/NNP', 0.55213463306427),
 ('국왕/NNG', 0.5427425503730774),
 ('일본/NNP', 0.5352097749710083),
 ('우리나라/NNG', 0.5296671986579895),
 ('중국/NNP', 0.5183144211769104),
 ('대마도/NNP', 0.5150614976882935),
 ('명나라/NNP', 0.5087485909461975),
 ('신라/NNP', 0.5077036023139954),
 ('개국/NNG', 0.5076214671134949),
 ('동방/NNG', 0.5046806931495667),
 ('본국/NNG', 0.5005097389221191),
 ('고구려/NNP', 0.47771331667900085),
 ('삼한/NNP', 0.4769449532032013),
 ('거란/NNP', 0.4754819869995117),
 ('대명/NNG', 0.47370511293411255),
 ('황조/NNP', 0.4699831008911133),
 ('청나라/NNP', 0.4684552550315857),
 ('태조/NNP', 0.4645532965660095),
 ('조/NNB', 0.46082115173339844),
 ('귀국/NNG', 0.4583689570426941)]

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('해동/NNP', topn=20)  # get other similar words

KeyError: "Key '해동/NNP' not present"

### 원문 Word2vec

In [ ]:
#분석용 피클파일 불러오기
# https://drive.google.com/file/d/1BCAZmBi3OK9GFtC2prab4wCRQtJaYUVq/view?usp=sharing
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1BCAZmBi3OK9GFtC2prab4wCRQtJaYUVq" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1BCAZmBi3OK9GFtC2prab4wCRQtJaYUVq" -o df_sent.pkl

In [ ]:
with open('df_sent.pkl', "rb") as fh:
  df_sent = pickle.load(fh)

In [ ]:
# vector : 차원, window : 앞뒤 단어 보는 크기, min_count : 분석에 포함할 최소 단어 빈도수, sg : 0이면 cbow, 1이면 skip-gram
v300_w5_min5000_sg = Word2Vec(sentences=df_sent['token'].to_list(), vector_size=300, window=5, min_count=300,sg=1,seed=2021, workers=1)
v300_w5_min5000_sg.save('ch_v300_w5_min300_sg.model') # 모델 저장. 모델 로드할때는 model= Word2Vec.load('v200_w3_min300_sg.model')

In [ ]:
v300_w5_min300_sg = Word2Vec.load('ch_v300_w5_min300_sg.model')

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('國', topn=20)  # get other similar words

[('邦', 0.49060723185539246),
 ('廷', 0.3014860451221466),
 ('朝', 0.29671192169189453),
 ('域', 0.2852965295314789),
 ('家', 0.28310567140579224),
 ('麗', 0.2802332639694214),
 ('疆', 0.27800172567367554),
 ('狄', 0.276493638753891),
 ('貴', 0.2671152353286743),
 ('大', 0.2667595446109772),
 ('搢', 0.2611580193042755),
 ('億', 0.25519484281539917),
 ('也', 0.2522425651550293),
 ('者', 0.25005918741226196),
 ('王', 0.24942584335803986),
 ('閥', 0.24924707412719727),
 ('寇', 0.24853019416332245),
 ('社', 0.2478121966123581),
 ('宗', 0.24748802185058594),
 ('捍', 0.24625685811042786)]

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('我', topn=20)

[('吾', 0.4932482838630676),
 ('余', 0.40199732780456543),
 ('予', 0.32671672105789185),
 ('朕', 0.30729252099990845),
 ('鮮', 0.2977280020713806),
 ('敝', 0.2936617136001587),
 ('人', 0.2914992570877075),
 ('賴', 0.28828781843185425),
 ('王', 0.28543445467948914),
 ('瀷', 0.28015753626823425),
 ('祖', 0.2762998342514038),
 ('皇', 0.2755458652973175),
 ('爾', 0.26780253648757935),
 ('矧', 0.2653608024120331),
 ('彼', 0.26424384117126465),
 ('汝', 0.2619032561779022),
 ('溘', 0.2569100856781006),
 ('寡', 0.25523895025253296),
 ('惟', 0.25349894165992737),
 ('饗', 0.25345274806022644)]

In [ ]:
v300_w5_min300_sg.wv.most_similar('吾', topn=20)

[('我', 0.49324825406074524),
 ('余', 0.48607683181762695),
 ('渠', 0.44041797518730164),
 ('汝', 0.41565221548080444),
 ('兄', 0.4037293493747711),
 ('瀷', 0.3959735631942749),
 ('弟', 0.3628588914871216),
 ('儕', 0.3415936529636383),
 ('其', 0.3364463746547699),
 ('姪', 0.32650044560432434),
 ('此', 0.321274071931839),
 ('爲', 0.3208879828453064),
 ('謂', 0.3201736807823181),
 ('予', 0.31980982422828674),
 ('君', 0.3190440237522125),
 ('則', 0.3160014748573303),
 ('但', 0.31013214588165283),
 ('生', 0.3090127110481262),
 ('人', 0.30574333667755127),
 ('父', 0.2988203465938568)]

In [ ]:
v300_w5_min300_sg.wv.most_similar('東', topn=20)

[('西', 0.5984089374542236),
 ('南', 0.5458069443702698),
 ('北', 0.4687977731227875),
 ('湖', 0.34696611762046814),
 ('瀛', 0.3141981065273285),
 ('海', 0.31271299719810486),
 ('球', 0.3096626102924347),
 ('萊', 0.3034631013870239),
 ('坡', 0.3008774220943451),
 ('淮', 0.30057740211486816),
 ('伽', 0.2989337146282196),
 ('江', 0.2839382290840149),
 ('山', 0.28217917680740356),
 ('邦', 0.2709294557571411),
 ('滄', 0.27080878615379333),
 ('迤', 0.26823994517326355),
 ('璿', 0.26807478070259094),
 ('嵩', 0.2678627371788025),
 ('州', 0.2661154866218567),
 ('嶠', 0.26283448934555054)]

In [ ]:
target_list = ['我', '吾', '東', '國']

In [ ]:
res = pd.DataFrame()
for target in target_list:
    res = pd.concat([res,pd.DataFrame(v300_w5_min300_sg.wv.most_similar(target, topn=20),columns=[target,'similarity'])],axis=1)  # get other similar words
res

,我,similarity,吾,similarity,東,similarity,國,similarity
0,吾,0.493248,我,0.493248,西,0.598409,邦,0.490607
1,余,0.401997,余,0.486077,南,0.545807,廷,0.301486
2,予,0.326717,渠,0.440418,北,0.468798,朝,0.296712
3,朕,0.307293,汝,0.415652,湖,0.346966,域,0.285297
4,鮮,0.297728,兄,0.403729,瀛,0.314198,家,0.283106
5,敝,0.293662,瀷,0.395974,海,0.312713,麗,0.280233
6,人,0.291499,弟,0.362859,球,0.309663,疆,0.278002
7,賴,0.288288,儕,0.341594,萊,0.303463,狄,0.276494
8,王,0.285434,其,0.336446,坡,0.300877,貴,0.267115
9,瀷,0.280158,姪,0.326500,淮,0.300577,大,0.266760


In [ ]:
res.to_excel('한국고전종합DB_원문_word2vec.xlsx')